# Лабораторная работа по теме "Metamorphic testing"

Ссылка на материалы занятия https://docs.google.com/presentation/d/1BvXMTDu0Go8mqeDxmdMKrxPe1pH4RJfZ/edit?usp=sharing&ouid=109129720852404525016&rtpof=true&sd=true

Литература

1) Barr E.T., Harman M., McMinn P., Shahbaz M., Yoo S. The oracle problem in software testing: A survey. IEEE transactions on software engineering. 41(5). 2014. P. 507-525. https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6963470

2) Tsong Yueh Chen, Fei-Ching Kuo, Huai Liu, Pak-Lok Poon, Dave Towey, T. H.
Tse, and Zhi Quan Zhou. 2018. Metamorphic Testing: A Review of Challenges
and Opportunities. 51, 1, Article 4 (Jan. 2018), 27 pages. https://doi.org/10.1145/3143561

3) M. Srinivasan, M. P. Shahri, I. Kahanda and U. Kanewala, "Quality Assurance of Bioinformatics Software: A Case Study of Testing a Biomedical Text Processing Tool Using Metamorphic Testing", 2018 IEEE/ACM 3rd International Workshop on Metamorphic Testing (MET), Gothenburg, 2018, pp. 26-33. https://arxiv.org/pdf/1802.07354.pdf


**Критерии оценки.**

Условная оценка - 10 баллов за лабораторную. Для получения зачёта достаточно набрать 6.



- Задание 1. (4 вопроса, 1 балл)
- Задание 2. (9 баллов)

## Задание 1. Теоретическая часть

Ответьте на следующие вопросы (*1 балл*).
1. Опишите суть проблемы тестового оракула.
2. Приведите примеры задач, для которых обычное тестирование с оракулом не подходит.
3. Перечислите методы, которые применяются для решения этой проблемы.
4. Дайте определение тестового инварианта (metamorphic relation).





Приведите свои ответы здесь:

1. 
2. 
3. 
4. 

## Задание 2. Разработка тестовых инвариантов

Дана модель для распознавания сущностей в тексте.
- Придумайте и реализуйте 2 теста с тестовым оракулом (обычные тесты с правильными ответами) (*1 балл*)
- Придумайте и реализуйте не менее 3 тестовых инвариантов (metamorphic relations) для её проверки - (*суммарно 6 баллов, теоретическое описание - по 1 баллу, реализация - по 1 баллу*)
- Сравните полученные тесты. В чем преимущества каждого из методов? В чём недостатки? (*2 балла*)

*Указание*. Можете воспользоваться идеями со слайда "Bio-entity recognition" или из статьи "Quality Assurance of Bioinformatics Software: A Case Study of Testing a Biomedical Text Processing Tool Using Metamorphic Testing" из списка литературы.

**Правила оформления.**

Для каждого инварианта необходимо описать
 - из каких предположений о модели он вытекает,
 - формальное описание (желательно с формулой),
 - запуск на 1-2 примерах тестовых данных.
 
Теоретическую часть можно оформить в ячейке markdown.

### Тестируемая система

Модель позволяет искать сущности в тексте. Ниже приведены примеры того, как можно с ней работать. 

In [ ]:
# ! pip install spacy==2.2.4

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# example of model usage
def render(text):
  
    nlp = spacy.load('en_core_web_sm') # model
  
    doc = nlp(text) # data processing

    # FYI all the properties
    props = [p for p in dir(doc.ents[0]) if p[0] != '_']
    print(props)

    # custom processing of the answer

    # get counts of entities
    ent_labels = [e.label_ for e in doc.ents]
    freq = dict()
    for l in ent_labels:
        freq[l] = ent_labels.count(l)
    print(freq)

    # get coordinates of entities
    coordinates = [e.start_char for e in doc.ents]
    print(coordinates)

    # render the answer
    displacy.render(doc, style='ent', jupyter=True)

Модель позволяет искать сущности в тексте. В ячейке ниже приведены примеры того, как можно с ней работать. 

In [ ]:
text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""

render(text)

['as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'lower_', 'merge', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'set_extension', 'similarity', 'start', 'start_char', 'string', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'upper_', 'vector', 'vector_norm', 'vocab']
{'ORG': 4, 'PRODUCT': 3}
[4, 84, 92, 124, 133, 167, 176]



### Пример оформления инварианта

Рассмотрим задачу поиска подстроки в строке. 

Предполагаем, что алгоритм должен находить все вхождения подстроки.

> **MR1.** Если в строке S найдёна некоторая подстрока s ровно k раз, то в строке SS она будет найдена не менее 2k раз (возможны нахождения на месте склейки строк).

Формально. Пусть S - строка, s - её подстрока, f(S,s) - определённое программой число вхождений s в S. Тогда f(SS,s) >= 2*f(S,s).


In [ ]:
import re
import unittest

# function for testing
def func_to_test(substr, string):
    return re.finditer(pattern=substr, string=string)


class TestStringMethods(unittest.TestCase):

    def test_with_oracle1(self):
        # input data
        big_string = 'abacaba'
        substr = 'aba'

        # correct answer
        right_answer = [0, 4]

        indices = func_to_test(substr, big_string)
        answer = [index.start() for index in indices]
        
        self.assertTrue(answer == right_answer)

    def test_metamorphic1(self):
        # input data
        big_string1 = 'abacab'
        big_string2 = big_string1 + big_string1
        substr = 'aba'

        # first answer
        indices = func_to_test(substr, big_string1)
        indices1 = [index.start() for index in indices]

        # second answer
        indices = func_to_test(substr, big_string2)
        indices2 = [index.start() for index in indices]

        # metamorphic relation
        self.assertTrue(2*len(indices1) <= len(indices2))

Ниже мы запускаем все объявленные тестовые случаи

In [ ]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


### Реализуйте собственные тестовые инварианты ниже

In [ ]:
class TestStringMethods(unittest.TestCase):
  
  def my_test_case():
    ...

[]
